In [91]:
# import libraries
import torch
from torch import nn
from torch.nn import functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [2]:
import sys
from datetime import timedelta
sys.path.append("../")
sys.path.append("../../")
import os
import random
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt
import datetime
from open_data import open_data, create_global_batch

In [3]:
PATH_CSV="/Users/clemencevast/Documents/Courses/MachineLearning/RadarTrafficData_files/Radar_Traffic_Counts.csv" #"/Users/iris/Documents/radar_deep/Radar_Traffic_Counts.csv"
radar_name=' CAPITAL OF TEXAS HWY / LAKEWOOD DR'

In [4]:
df1=open_data(PATH_CSV, direction="NB", radar=radar_name, year=2018)

In [5]:
batch_df=create_global_batch(df1, window_x_day=6, window_label_day=1, gap_acquisition=1, tot_len_day=35)

Building batch 0 
 x begin 2018-01-02 label begin 2018-01-08 end period 2018-01-30 
Building batch 1 
 x begin 2018-01-03 label begin 2018-01-09 end period 2018-01-30 
Building batch 2 
 x begin 2018-01-04 label begin 2018-01-10 end period 2018-01-30 
Building batch 3 
 x begin 2018-01-05 label begin 2018-01-11 end period 2018-01-30 
Building batch 4 
 x begin 2018-01-06 label begin 2018-01-12 end period 2018-01-30 
Building batch 5 
 x begin 2018-01-07 label begin 2018-01-13 end period 2018-01-30 
We do not have all the dates for the time period in label , 1 []
Building batch 5 
 x begin 2018-01-08 label begin 2018-01-14 end period 2018-01-30 
We do not have all the dates for the time period in x , 6 [datetime.date(2018, 1, 8) datetime.date(2018, 1, 9)
 datetime.date(2018, 1, 10) datetime.date(2018, 1, 11)
 datetime.date(2018, 1, 12)]
Building batch 5 
 x begin 2018-01-09 label begin 2018-01-15 end period 2018-01-30 
We do not have all the dates for the time period in x , 6 [datetime.

In [8]:
batch_df

,vol_data_x,vol_label_y
0,"[22, 13, 21, 21, 15, 11, 19, 5, 14, 6, 5, 8, 4...","[24, 21, 18, 10, 14, 10, 9, 12, 7, 10, 9, 9, 5..."
1,"[33, 31, 20, 22, 13, 13, 13, 9, 3, 13, 4, 6, 1...","[66, 40, 40, 22, 28, 22, 13, 9, 10, 14, 17, 7,..."
2,"[32, 31, 25, 29, 29, 10, 21, 17, 13, 13, 5, 8,...","[36, 32, 28, 26, 23, 20, 14, 21, 8, 9, 11, 9, ..."
3,"[33, 37, 28, 19, 26, 16, 13, 14, 14, 5, 8, 6, ...","[48, 32, 36, 28, 32, 23, 8, 22, 17, 7, 21, 15,..."
4,"[59, 57, 56, 46, 30, 32, 19, 16, 19, 14, 17, 1...","[54, 36, 34, 30, 28, 19, 26, 17, 10, 8, 13, 14..."
5,"[100, 101, 95, 95, 109, 83, 91, 82, 124, 109, ...","[24, 26, 16, 17, 10, 8, 10, 11, 11, 10, 10, 5,..."
6,"[13, 5, 14, 15, 9, 4, 6, 1, 4, 1, 5, 2, 4, 6, ...","[33, 38, 45, 30, 25, 10, 13, 15, 4, 8, 4, 9, 1..."
7,"[34, 29, 28, 26, 17, 14, 15, 5, 6, 11, 8, 7, 5...","[38, 38, 26, 21, 26, 22, 18, 11, 2, 4, 8, 6, 1..."
8,"[32, 25, 20, 25, 20, 19, 18, 8, 6, 12, 10, 9, ...","[35, 35, 35, 23, 23, 14, 13, 13, 8, 6, 9, 9, 6..."
9,"[56, 54, 41, 39, 27, 28, 32, 17, 18, 12, 18, 1...","[38, 34, 32, 23, 29, 15, 22, 16, 12, 12, 20, 9..."


In [64]:
class RadarDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe["vol_data_x"].values
        self.target = dataframe["vol_label_y"].values
        
    def __len__(self): # total number of samples
        return len(self.data)

    def __getitem__(self, index): # how to preprocess the files (index = iterator on samples)
        sample = self.data[index]
        label = self.target[index]
        return sample, label

In [73]:
class RadarCollate(object):
    """Function object used as a collate function for DataLoader."""

    def __init__(self,):
        pass
        
    def _collate_fn(self, batch):
        data_list, label_list = [], []
        for _data, _label in batch:
            data_list.append(_data)
            label_list.append(_label)
        return torch.Tensor(data_list), torch.LongTensor(label_list)
    
    def __call__(self, batch):
        return self._collate_fn(batch)


In [86]:
# sample_rate = 16000
batch_size = 1

In [87]:
fullset = RadarDataset(dataframe=batch_df)

collate_fn = RadarCollate()
trainloader = DataLoader(trainset,batch_size=batch_size, collate_fn=collate_fn,shuffle=True)


In [88]:
c= 0
for x in trainloader:
    print("batch number {}".format(c))
    print(x[0].shape, x[1].shape, "\n")
    c+=1

batch number 0
torch.Size([1, 574]) torch.Size([1, 96]) 

batch number 1
torch.Size([1, 574]) torch.Size([1, 96]) 

batch number 2
torch.Size([1, 574]) torch.Size([1, 42]) 

batch number 3
torch.Size([1, 574]) torch.Size([1, 96]) 

batch number 4
torch.Size([1, 572]) torch.Size([1, 96]) 

batch number 5
torch.Size([1, 574]) torch.Size([1, 96]) 

batch number 6
torch.Size([1, 521]) torch.Size([1, 94]) 

batch number 7
torch.Size([1, 574]) torch.Size([1, 96]) 

batch number 8
torch.Size([1, 574]) torch.Size([1, 96]) 

batch number 9
torch.Size([1, 576]) torch.Size([1, 94]) 

batch number 10
torch.Size([1, 576]) torch.Size([1, 94]) 

batch number 11
torch.Size([1, 572]) torch.Size([1, 96]) 

batch number 12
torch.Size([1, 574]) torch.Size([1, 96]) 

batch number 13
torch.Size([1, 574]) torch.Size([1, 96]) 

batch number 14
torch.Size([1, 574]) torch.Size([1, 94]) 

batch number 15
torch.Size([1, 574]) torch.Size([1, 96]) 

batch number 16
torch.Size([1, 574]) torch.Size([1, 96]) 

batch n

In [93]:
input_dim = 574    
hidden_dim = 256
layer_dim = 3
output_dim = 96
seq_dim = 1

lr = 0.05
n_epochs = 10
iterations_per_epoch = len(trainloader)
best_acc = 0